In [67]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
from matplotlib import rcParams
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
rcParams['figure.figsize'] = (12,6)

import numpy as np
import pandas as pd
import seaborn as sns

import os
import torch
from torch import nn
from torchvision import models

from torchsummary import summary
from ray import tune

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [135]:
exp_config = dict(
    #### model related
    device="cuda",
#    network=tune.grid_search(["vgg19_bn_kw", "vgg19_bn"]),
    network="VGG19",
    num_classes=10,
    model=tune.grid_search(["BaseModel", "SparseModel", "SET", "DSNN"]),
    # model=tune.grid_search(["SET", "DSNN", "DSNN_Flip", "DSNN_Correct"]),
    dataset_name="CIFAR10",
    augment_images=True,
    stats_mean=(0.4914, 0.4822, 0.4465),
    stats_std=(0.2023, 0.1994, 0.2010),
    data_dir="~/nta/datasets",
    #### optimizer related
    optim_alg="SGD",
    momentum=0.9,
    learning_rate=0.01,
    lr_scheduler="MultiStepLR",
    lr_milestones=[250,290],
    lr_gamma=0.10,
    debug_weights=True,
    #### sparse related
    epsilon=100,
    start_sparse=1,
    end_sparse=None, 
    debug_sparse=True,
    flip=False,
    weight_prune_perc=0.3,
    grad_prune_perc=0,
    percent_on=0.3,
    boost_strength=1.4,
    boost_strength_factor=0.7,
    test_noise=True,
    noise_level=0.1,
    kwinners=True # moved to a parameter
)


In [141]:
from networks import VGG19
model = VGG19(exp_config)

<bound method VGG19._kwinners of VGG19()>
<class 'nupic.torch.modules.k_winners.KWinners2d'>
KWinners2d(channels=10, n=0, percent_on=0.3, boost_strength=1.4, duty_cycle_period=1000)


#### how to replace RELUs with another non-linear activation function

In [33]:
model = models.vgg19_bn()

In [7]:
model.features

Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace)
  (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (5): ReLU(inplace)
  (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (9): ReLU(inplace)
  (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (12): ReLU(inplace)
  (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (14): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): BatchNor

In [61]:
model = models.vgg19_bn()
config = {
    'percent_on': 0.3,
    'boost_strength': 1.4,
    'boost_strength_factor': 0.7
}
new_features = []
for layer in model.features:
    # remove max pooling
    if isinstance(layer, nn.MaxPool2d):
        pass
    elif isinstance(layer, nn.Conv2d):
        new_features.append(layer)
        last_conv_out_channels = layer.out_channels
    # switch ReLU to kWinners2d
    elif isinstance(layer, nn.ReLU):
        new_features.append(KWinners2d(
            channels=last_conv_out_channels,
            percent_on=config['percent_on'],
            boost_strength=config['boost_strength'],
            boost_strength_factor=config['boost_strength_factor']))
    # otherwise add it as normal
    else:
        new_features.append(layer)
        
model.features = nn.Sequential(*new_features)

In [62]:
model

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): KWinners2d(channels=64, n=0, percent_on=0.3, boost_strength=1.4, duty_cycle_period=1000)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): KWinners2d(channels=64, n=0, percent_on=0.3, boost_strength=1.4, duty_cycle_period=1000)
    (6): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): KWinners2d(channels=128, n=0, percent_on=0.3, boost_strength=1.4, duty_cycle_period=1000)
    (9): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (10): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): KWi

In [63]:
summary(model, input_size=(3,32,32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           1,792
       BatchNorm2d-2           [-1, 64, 32, 32]             128
        KWinners2d-3           [-1, 64, 32, 32]               0
            Conv2d-4           [-1, 64, 32, 32]          36,928
       BatchNorm2d-5           [-1, 64, 32, 32]             128
        KWinners2d-6           [-1, 64, 32, 32]               0
            Conv2d-7          [-1, 128, 32, 32]          73,856
       BatchNorm2d-8          [-1, 128, 32, 32]             256
        KWinners2d-9          [-1, 128, 32, 32]               0
           Conv2d-10          [-1, 128, 32, 32]         147,584
      BatchNorm2d-11          [-1, 128, 32, 32]             256
       KWinners2d-12          [-1, 128, 32, 32]               0
           Conv2d-13          [-1, 256, 32, 32]         295,168
      BatchNorm2d-14          [-1, 256,

In [42]:
from nupic.torch.modules import KWinners2d
new_features = []
for layer in model.features:
    if isinstance(layer, nn.MaxPool2d):
        pass
    elif isinstance(layer, nn.ReLU):
        new_features.append(KWinners2d(0, percent_on=0.3))
    else:
        new_features.append(layer)

In [46]:
model.features = nn.Sequential(*new_features)
summary(model, input_size=(3,32,32))

RuntimeError: The size of tensor a (64) must match the size of tensor b (0) at non-singleton dimension 1

In [44]:
model

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): KWinners2d(channels=0, n=0, percent_on=0.3, boost_strength=1.0, duty_cycle_period=1000)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): KWinners2d(channels=0, n=0, percent_on=0.3, boost_strength=1.0, duty_cycle_period=1000)
    (6): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): KWinners2d(channels=0, n=0, percent_on=0.3, boost_strength=1.0, duty_cycle_period=1000)
    (9): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (10): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): KWinner

In [31]:
from nupic.torch.modules import KWinners
for idx, layer in enumerate(model.features):
    
    if isinstance(layer, nn.ReLU):
        model.features[idx] = KWinners(0, 0.3)
    elif is

In [30]:
model

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): KWinners(n=0, percent_on=0.3, boost_strength=1.0, duty_cycle_period=1000)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): KWinners(n=0, percent_on=0.3, boost_strength=1.0, duty_cycle_period=1000)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): KWinners(n=0, percent_on=0.3, boost_strength=1.0, duty_cycle_period=1000)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, 

In [10]:
dir(m)

['__call__',
 '__class__',
 '__constants__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_apply',
 '_backend',
 '_backward_hooks',
 '_buffers',
 '_forward_hooks',
 '_forward_pre_hooks',
 '_get_name',
 '_load_from_state_dict',
 '_load_state_dict_pre_hooks',
 '_modules',
 '_named_members',
 '_parameters',
 '_register_load_state_dict_pre_hook',
 '_register_state_dict_hook',
 '_slow_forward',
 '_state_dict_hooks',
 '_tracing_name',
 '_version',
 'add_module',
 'apply',
 'bias',
 'buffers',
 'children',
 'cpu',
 'cuda',
 'double',
 'dump_patches',
 'eval',
 'extra_repr',
 'float',
 'forward',
 'half',
 'in_features',
 'load_state_dict',
 'modules',
 

In [ ]:
class cauchy_activation(nn.Module):
    def __init__(self):
        super(cauchy_activation, self).__init__()
        
    def activation(self, inp):
        return pdf_cauchy_distribution(self.inp)